# Malicious URL Pipeline

In [0]:
#Install Python Packages
%pip install tldextract python-whois

from urllib.parse import urlparse
import tldextract
import whois
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType, BooleanType
import pandas as pd

In [0]:
# Secrets to access storage
processed_sas_token = dbutils.secrets.get(scope="trecombs", key="ProcessedSASToken")
sas_url = dbutils.secrets.get(scope="trecombs", key="sasurlvalue")
# Define storage account + container
storage_account = "formula1dl122334"
container_name = "processed"

# -----------------------------
# 3️⃣ Configure Spark with SAS token
# -----------------------------
spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account}.blob.core.windows.net",
    processed_sas_token
)
spark.conf.set(
    "fs.azure.account.key.formula1dl122334.dfs.core.windows.net",
    dbutils.secrets.get(scope="trecombs", key="azurestoragekey")
)

In [0]:
# Load CSV into Spark using pandas
sas_url = dbutils.secrets.get(scope="trecombs", key="sas_url")
pdf = pd.read_csv("sas_url")

# Convert Pandas DataFrame to Spark DataFrame
df = spark.createDataFrame(pdf)

# Helpers for URL parsing
def extract_domain(url: str) -> str:
    if not url:
        return None
    try:
        parsed = urlparse(url if "://" in url else f"http://{url}")
        host = parsed.netloc if parsed.netloc else parsed.path
        ext = tldextract.extract(host)
        if ext.domain and ext.suffix:
            return f"{ext.domain}.{ext.suffix}".lower()
        elif host:
            return host.lower()
        return None
    except Exception:
        return None

def extract_tld(url: str) -> str:
    if not url:
        return None
    try:
        parsed = urlparse(url if "://" in url else f"http://{url}")
        host = parsed.netloc if parsed.netloc else parsed.path
        ext = tldextract.extract(host)
        return ext.suffix.lower() if ext.suffix else None
    except Exception:
        return None

def whois_owner(domain: str) -> str:
    if not domain:
        return None
    try:
        w = whois.whois(domain)
        for key in ["org", "organization", "registrant_name", "name", "registrant_organization"]:
            val = w.get(key)
            if isinstance(val, list):
                val = val[0] if val else None
            if val:
                return str(val)
        registrar = w.get("registrar")
        if registrar:
            return str(registrar)
    except Exception:
        pass
    return None

def count_e_in_domain(domain: str) -> int:
    return domain.count("e") if domain else 0

def has_A_and_T(domain: str) -> bool:
    if not domain:
        return False
    d = domain.upper()
    return ("A" in d) and ("T" in d)

# UDFs
extract_domain_udf = F.udf(extract_domain, StringType())
extract_tld_udf    = F.udf(extract_tld, StringType())
owner_udf          = F.udf(whois_owner, StringType())
count_e_udf        = F.udf(count_e_in_domain, IntegerType())
has_AT_udf         = F.udf(has_A_and_T, BooleanType())


# Transform Data
enriched = (df
    .withColumn("domain", extract_domain_udf(F.col("url")))
    .withColumn("tld", extract_tld_udf(F.col("url")))
    .withColumn("owner", owner_udf(F.col("domain")))        
    .withColumn("e_count", count_e_udf(F.col("domain")))
    .withColumn("has_A_and_T", has_AT_udf(F.col("domain")))
)

display(enriched.limit(10))

# Count malware/phishing with A and T

malware_phishing_AT = (enriched
    .filter( (F.upper(F.col("type")).isin("MALWARE", "PHISHING")) & (F.col("has_A_and_T") == True) )
    .count()
)

print(f"Records with A and T in domain that are malware or phishing: {malware_phishing_AT}")

table_name = "malicious_urls_project_catalog.enriched_data.malicious_urls_enriched"

# Drop/create table using CREATE OR REPLACE TABLE
enriched.createOrReplaceTempView("temp_enriched")  # create a temp view

spark.sql(f"""
CREATE OR REPLACE TABLE {table_name}
USING DELTA
AS
SELECT * FROM temp_enriched
""")


url,type,domain,tld,owner,e_count,has_A_and_T
br-icloud.com.br,phishing,br-icloud.com.br,com.br,Toweb Brasil LTDA EPP,0,false
mp3raid.com/music/krizz_kaliko.html,benign,mp3raid.com,com,"GoDaddy.com, LLC",0,false
bopsecrets.org/rexroth/cr/1.htm,benign,bopsecrets.org,org,REDACTED,2,false
http://www.garage-pirenne.be/index.php?option=com_content&view=article&id=70&vsig70_0=15,defacement,garage-pirenne.be,be,Above.com Domain Privacy,4,false
http://adventure-nicaragua.net/index.php?option=com_mailto&tmpl=component&link=aHR0cDovL2FkdmVudHVyZS1uaWNhcmFndWEubmV0L2luZGV4LnBocD9vcHRpb249Y29tX2NvbnRlbnQmdmlldz1hcnRpY2xlJmlkPTQ3OmFib3V0JmNhdGlkPTM2OmRlbW8tYXJ0aWNsZXMmSXRlbWlkPTU0,defacement,adventure-nicaragua.net,net,Above.com Pty Ltd.,3,true
http://buzzfil.net/m/show-art/ils-etaient-loin-de-s-imaginer-que-le-hibou-allait-faire-ceci-quand-ils-filmaient-2.html,benign,buzzfil.net,net,REDACTED FOR PRIVACY,1,false
espn.go.com/nba/player/_/id/3457/brandon-rush,benign,go.com,com,"CSC Corporate Domains, Inc.",0,false
yourbittorrent.com/?q=anthony-hamilton-soulife,benign,yourbittorrent.com,com,Whois Privacy Corp.,1,false
http://www.pashminaonline.com/pure-pashminas,defacement,pashminaonline.com,com,REDACTED FOR PRIVACY,1,false
allmusic.com/album/crazy-from-the-heat-r16990,benign,allmusic.com,com,"All Media Network, LLC",0,false


Records with A and T in domain that are malware or phishing: 42


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

The following query calculates how many domains exist per type and what percentage each type represents of the total dataset.

In [0]:
%sql
SELECT 
    type,
    COUNT(DISTINCT domain) AS domain_count,
    ROUND(
        COUNT(DISTINCT domain) * 100.0 / SUM(COUNT(DISTINCT domain)) OVER (), 
        2
    ) AS percentage_of_total
FROM malicious_urls_project_catalog.enriched_data.malicious_urls_enriched
GROUP BY type
ORDER BY domain_count DESC;


type,domain_count,percentage_of_total
benign,752,68.80
defacement,247,22.60
phishing,69,6.31
malware,25,2.29


The following query assigns a severity rank to each URL based on its type and orders the results by severity. Limit 50 rows

In [0]:
%sql
SELECT
    url,
    type,
    domain,
    CASE 
        WHEN type = 'malware' THEN 'High'
        WHEN type = 'defacement' THEN 'Medium High'
        WHEN type = 'phishing' THEN 'Medium'
        WHEN type = 'benign' THEN 'Low'
    END AS severity_rank
FROM malicious_urls_project_catalog.enriched_data.malicious_urls_enriched
where type is not null and domain is not null
ORDER BY 
    CASE 
        WHEN type = 'malware' THEN 1
        WHEN type = 'defacement' THEN 2
        WHEN type = 'phishing' THEN 3
        WHEN type = 'benign' THEN 4
    END
    LIMIT 50





url,type,domain,severity_rank
http://ak.imgfarm.com/images/nocache/vicinio/installers/205320000.S10570.1/507981-150710122501-S10570.1/VideoDownloadConvertAuto.exe_0,malware,imgfarm.com,High
http://www.824555.com/app/member/SportOption.php?uid=guest&langx=gb,malware,824555.com,High
http://9779.info/%E5%85%AD%E4%B8%80%E5%BF%AB%E4%B9%90%E7%B2%98%E8%B4%B4%E7%94%BB/,malware,9779.info,High
http://9779.info/%E6%8A%98%E7%BA%B8%E6%89%87%E5%AD%90%E6%8B%BC%E8%B4%B4%E7%94%BB/,malware,9779.info,High
http://www.0068555.com/cl/?module=System&method=LiveTop&args=livehall,malware,0068555.com,High
http://9779.info/%E5%84%BF%E7%AB%A5%E7%AB%8B%E4%BD%93%E7%BA%B8%E8%B4%B4%E7%94%BB/,malware,9779.info,High
http://9779.info/%E5%B9%BC%E5%84%BF%E5%9B%AD%E7%9A%B1%E7%BA%B9%E7%BA%B8%E7%B2%98%E8%B4%B4%E7%94%BB/,malware,9779.info,High
http://9779.info/%E6%A0%91%E5%8F%B6%E7%B2%98%E8%B4%B4%E7%94%BB/,malware,9779.info,High
http://www.easycaptchas.com/2F3F28039730C12A5C112FE0D5FEFA2E.captcha?width=132&height=42,malware,easycaptchas.com,High
http://9779.info/%E5%8F%A4%E4%BB%A3%E4%BA%8C%E5%8D%81%E5%9B%9B%E5%AD%9D%E5%B8%83%E8%B4%B4%E7%94%BB/,malware,9779.info,High
